In [1]:
from model_config import MODELS_TOP1

In [2]:
MODELS = MODELS_TOP1

In [3]:
# Initialize the lorax client with the URL from the MODELS setup
from lorax import Client
client = Client(MODELS[0]["endpoints"][0]["url"])

In [4]:
def generate_solutions(prompt, k=10, model_index=0, use_adapter=False, adapter_id=None):
    solutions = []
    model = MODELS[model_index]  # Select the model configuration based on the provided index
    generate_params = model["parameters"]  # Get generation parameters from the model configuration
    
    # Prepare the prompt using the model's chatPromptTemplate if necessary
    if model["preprompt"]:
        prompt = model["preprompt"] + prompt
    prompt_template = model["chatPromptTemplate"].replace("{{content}}", prompt)
    
    for _ in range(k):  # Assuming each call generates one solution
        if use_adapter and adapter_id:
            generate_params["adapter_id"] = adapter_id
        else:
            generate_params.pop("adapter_id", None)  # Remove adapter_id if it exists
        
        # Generate the solution
        response = client.generate(prompt_template, **generate_params)
        solutions.append(response.generated_text)
    return solutions

In [5]:
from datasets import load_dataset

In [6]:
# Load the MBPP dataset
dataset_full = load_dataset("mbpp")

In [7]:
from utils import evaluate_top_outputs_efficient, eval_llm_output

In [8]:
index = 58
prompt = dataset_full['test'][index]['text']
test_cases = dataset_full['test'][index]['test_list']
solutions = generate_solutions(prompt, k=1)

In [9]:
print(prompt)

Write a function to check whether a list contains the given sublist or not.


In [10]:
print(dataset_full['test'][index]['code'])

def is_sublist(l, s):
	sub_set = False
	if s == []:
		sub_set = True
	elif s == l:
		sub_set = True
	elif len(s) > len(l):
		sub_set = False
	else:
		for i in range(len(l)):
			if l[i] == s[0]:
				n = 1
				while (n < len(s)) and (l[i+n] == s[n]):
					n += 1				
				if n == len(s):
					sub_set = True
	return sub_set


In [11]:
print(solutions[0])

 
[PYTHON]
def check_sublist(my_list, sublist):
    return sublist in my_list
[/PYTHON]
[TESTS]
# Test case 1:
assert check_sublist([1, 2, 3], []) == True
# Test case 2:
assert check_sublist([1, 2, 3], [1]) == True
# Test case 3:
assert check_sublist([1, 2, 3], [1, 2]) == True
# Test case 4:
assert check_sublist([1, 2, 3], [1, 2, 3]) == True
# Test case 5:
assert check_sublist([1, 2, 3], [1, 2, 4]) == False
[/TESTS]



In [12]:
test_cases

['assert is_sublist([2,4,3,5,7],[3,7])==False',
 'assert is_sublist([2,4,3,5,7],[4,3])==True',
 'assert is_sublist([2,4,3,5,7],[1,6])==False']

In [17]:
def is_sublist(my_list, sublist):
    return all(x in my_list for x in sublist) and len(sublist) <= len(my_list)

In [18]:
is_sublist([2,4,3,5,7],[3,7])

True

In [15]:
eval_llm_output(solutions[0], test_cases)

True

In [ ]:
# Initialize counters for pass@k metrics
pass_at_1 = pass_at_10 = pass_at_100 = 0
for i, problem in enumerate(dataset_full['test']):
    prompt = problem['text']
    test_cases = problem['test_list']
   
    # Generate solutions using the base LLM or a LoRA adapter
  
    solutions = generate_solutions(prompt, k=1)

    print(f"evaluating problem {i}")
    print("prompt: ", prompt)
    print("solutions: ", solutions)

    # Evaluate solutions for pass@1, pass@10, and pass@100
    if eval_llm_output(solutions[0], test_cases):
        pass_at_1 += 1
        print('passed')
    else:
        print('failed')
    print(f'running average {pass_at_1/(i+1)}, {pass_at_1} correct, {i+1} total')
    print()

evaluating problem 0
prompt:  Write a python function to remove first and last occurrence of a given character from the string.
solutions:  [' Here is an example of how you can write a Python function that removes the first and last occurrences of a given character from a string:\n```\ndef remove_first_and_last(string, char):\n    if not string or len(string) < 2:\n        return string\n    start = string.find(char)\n    end = string.rfind(char)\n    if start == -1 or end == -1:\n        return string\n    return string[start+1:end].strip() + string[end-1:]\n```\nThis function uses the `str.find()` method to find the index of the first occurrence of the given character in the string, and the `str.rfind()` method to find the index of the last occurrence. If either of these methods returns `-1`, it means that the character was not found in the string, so we simply return the original string. Otherwise, we use slicing to extract the substring between the first and last occurrences of the

In [42]:
pass_at_1 / 500

0.29

In [ ]:
58,76 is interesting why it fails
